## 实验要求
### 截止日期：12月15日
作业的提交格式参考之前的说明，提交到18329300691@163.com
### 基本要求
a)	基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
b)	基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；
### 中级要求
a)  对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
b)	对测试集Watermelon-test2进行预测，输出分类精度；
### 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。



## 决策树的划分
- 决策树主要分为三种：
	ID3，C4.5和CART，它们分别对应的**特征选择准则**是信息增益（ID3），信息增益比（C4.5）和基尼指数（CART）。
	它们决定当前选择哪个特征进行数据划分，使得样本在当下能够被最大程度的划分。
- 对于离散变量，选定**属性**分类即可；
- 对于连续变量，需要选定**划分点**。
- CART和C4.5支持数据特征为**连续分布**时的处理，能够完成对连续属性的离散化处理，主要通过二元切分的方式来处理连续型变量，这个分裂点的选择原则是使得划分后的子树中的“混乱程度”降低。

In [ ]:
import numpy as np 
import pandas as pd
import math
import copy
def read(name):
    df = pd.read_csv(name, error_bad_lines = False, encoding = 'gbk')
    temp = np.array(df).tolist()
    for i in temp:
        i.pop(0)
    return temp

train1 = read("D:\dasanshang\jiqixuexi\p6\Watermelon-train1.csv")
train2 = read("D:\dasanshang\jiqixuexi\p6\Watermelon-train2.csv")
test1 = read("D:\dasanshang\jiqixuexi\p6\Watermelon-test1.csv")
test2 = read("D:\dasanshang\jiqixuexi\p6\Watermelon-test2.csv")

## 首先进行数据的导入

In [3]:
def inport(data):
    dic = {}
    for i in data:
        current = i[-1] #取出最后的结果
        if current not in dic.keys(): 
            dic[current] = 1 #创建一个新的类别
        else:
            dic[current] += 1 #原有类别+1
    result = 0.0
    for key in dic:
        pro = float(dic[key]) / len(data)
        result -= pro * math.log(pro,2) 
    return result, dic

## 计算信息增益
- 统计某结果数据发生的频率，每项的信息以字典的形似存储
- 计算信息熵
- 返回信息熵和分类信息

In [4]:
def fen(data, index, kind):
    ls = []
    for temp in data:
        if temp[index] == kind:
            t = temp[0: index]
            t = t + temp[index + 1: ]
            ls.append(t)
    return ls

- 将数据按照某一种类的属性重新分类，并将该行属性删除
- 计算信息熵和信息增益并返回最优的分类方式，对数据的每项指标做以下的过程
    1. 抽取该项数据的所有信息
    2. 按照该项数据的类别信息将数据集划分成多个子数据集
    3. 计算每个数据集的信息熵
    5. 计算该项数据的信息增益
    6. 根据信息增益选择最好的分类项目，返回该项目的类别号

In [5]:
def zuijiatezhengfenli(data):
    base, mm= inport(data) #原始的信息熵
    best = 0
    bestindex = -1
    for i in range(len(data[0]) - 1):
        #抽取该列数据的所有信息
        ls = [index[i] for index in data]
        feture = set(ls) 
        #计算该列的信息增益
        temp = 0.0
        for value in feture:
            datatemp = split(data, i, value)
            prob = len(datatemp) / float(len(data))
            t, mm = information(datatemp)
            temp += prob * t
        infoGain = base - temp
        #根据信息增益挑选 
        if infoGain > best:
            best = infoGain
            bestindex = i
    return bestindex 

## ID3算法
- ID3算法的核⼼思想应用信息增益准则作为标准,介绍信息增益之前首先介绍一下信息熵和条件熵： 
- 熵（entropy）概念：
	    1948年，香农提出了“信息熵”的概念。在信息论与概率统计中，熵是表示随机变量不确定性的量。X是⼀个取值为有限个的离散随机变量，
$$ H(X)=-\sum_{i=1}^{n} p\left(x_{i}\right) \log p\left(x_{i}\right)$$ 
$𝐻(𝑋)$就被称作随机变量𝑋的熵，它表示随机变量不确定的度量。熵取值越大，随机变量不确定性越大。当随机变量为均匀分布时，熵最大。当某一状态概率取值为1时，熵的值为零。

### ID3算法-条件熵和信息增益
- 条件熵 $𝐻(𝑌∣𝑋)$ ：
	表示在已知随机变量𝑋的条件下随机变量𝑌的不确定性，定义为给定𝑋条件下𝑌的条件概率分布的熵对𝑋的数学期望:
$$H(Y \mid X)=\sum_{x} p(x) H(Y \mid X=x) =-\sum_{x} p(x) \sum_{y} p(y \mid x) \log p(y \mid x)$$

- 特征𝐴对数据集𝐷的信息增益就是熵$𝐻(𝐷)$与条件熵$𝐻(𝐷|𝐴)$之差:
$$𝐻(𝐷)−𝐻(𝐷∣𝐴)$$

	表示已知特征𝐴的信息而使得数据集𝐷的信息不确定减少的程度。信息增益越大的特征代表其具有更强的分类能力，所以我们就要**选择能够使数据的不确定程度减少最多的特征**，也就是信息增益最大的特征。

### ID3算法-停止条件
- 决策树的生成:

	从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
    
    
- 迭代停止条件：
  1. 当前结点所有样本都属于同⼀类别；
  2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
  3. 达到最大树深；
  4. 达到叶子结点的最小样本数；

In [6]:
def fen1(data, labels):
    typelist = [index[-1] for index in data] #取出该数据集的分类
    nothing, typecount = information(data) #计算出类别种类以及数量
    if len(typecount) == 1: #如果只有一个类别
        return typelist[0]
    bestindex = zuijiatezhengfenli(data)  # 最优划分属性的索引
    bestlabel = labels[bestindex]
    Tree = {bestlabel: {}}
    temp = labels[:]
    del (temp[bestindex])  # 已经选择的特征不再参与分类，将该类别删除
    feature = [example[bestindex] for example in data]
    unique = set(feature)  # 该属性所有可能取值，也就是节点的分支
    for i in unique:  
        temp = temp[:] 
        Tree[bestlabel][i] = classify1(split(data, bestindex, i), temp)
    return Tree

# 递归构建$ID3$决策树
- 决策树的生成
	1. 从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	2. 在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
- 迭代停止条件
    1. 当前结点所有样本都属于同⼀类别；
    2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
    3. 达到最大树深；
    4. 达到叶子结点的最小样本数；
- 具体实现
    1. 首先取出该数据集的类别信息
    2. 统计处类别信息以及数量，如果只有一个类别，返回该类别
    3. 计算最优划分的索引
    4. 初始化子树
    5. 当前已经选择的特征不再参与分类，将该特征删除
    6. 计算剩余特征的集合
    7. 对于每个分支，进行递归
    8. 返回值为子树


In [7]:
def ce1(testdata, tree, labels):
    firstStr = list(tree.keys())[0]
    secondDict = tree[firstStr]
    featIndex = labels.index(firstStr)
    result = ''
    for key in list(secondDict.keys()): 
         if testdata[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':  # 该分支不是叶子节点，递归
                result = run1(testdata, secondDict[key], labels)
            else:
                result = secondDict[key]
    return result

def shengc(train, test, labels):
    ls = []
    tree = fen2(train, labels)
    print("生成决策树如下:", tree)
    for index in test:
        ls.append(run1(index, tree, labels))
    return ls

## 对测试集利用$ID3$决策树进行分类
- 取出当前树的根节点
- 利用跟节点信息查询当前输入数据输入内容
- 如果查询出来的分支是叶节点，返回该值
- 如果不是叶节点，递归查询子树

In [8]:
labels1 = ['色泽', '根蒂', '敲声', '纹理', '好瓜']
result1 = shengc(train1, test1, labels1)

生成决策树如下: {'纹理': {'清晰': {'根蒂': {'稍蜷': '是', '蜷缩': '是', '硬挺': '否'}}, '模糊': '否', '稍糊': {'色泽': {'青绿': '否', '浅白': '否', '乌黑': {'敲声': {'浊响': '是', '沉闷': '否'}}}}}}


## 决策树的剪枝
- 决策树很容易出现**过拟合现象**。原因在于学习时完全考虑的是如何提⾼对训练数据的正确分类从⽽构建出过于复杂的决策树。
- 解决这个问题的方法称为**剪枝**，即对已生成的树进行简化。具体地，就是从已生成的树上裁剪掉⼀些子树或叶节点，并将其根节点或父节点作为新的叶节点。 
- 决策树的剪枝基本策略有**预剪枝 (Pre-Pruning)** 和 **后剪枝 (Post-Pruning)**
   - **预剪枝**：是根据⼀些原则**极早的停止树增长**，如树的深度达到用户所要的深度、节点中样本个数少于用户指定个数、不纯度指标下降的幅度小于用户指定的幅度等。 
   - **后剪枝**：是通过在完全生长的树上剪去分枝实现的，通过删除节点的分支来剪去树节点。是在生成决策树之后**自底向上**的对树中所有的非叶结点进⾏逐一考察 。

In [11]:
def prune(tree, testdata, testlabels):
    firstStr = list(tree.keys())[0]
    secondDict = tree[firstStr]
    
    # 判断是否为叶子节点，如果是叶子节点，则进行剪枝操作
    if type(secondDict).__name__ != 'dict':
        return secondDict
    
    featIndex = testlabels.index(firstStr)
    result = ''
    for key in list(secondDict.keys()):
        if testdata[featIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                result = prune(secondDict[key], testdata, testlabels)
            else:
                result = secondDict[key]
    
    # 基于剪枝后的子树进行后剪枝操作
    if all(val == result for val in secondDict.values()):
        return result
    else:
        return tree

# 使用剪枝后的决策树进行分类
def get_pruned_result(train, test, labels):
    ls = []
    tree = classify1(train, labels)
    print("生成决策树如下:", tree)
    pruned_tree = prune(tree, test, labels)
    print("剪枝后的决策树如下:", pruned_tree)
    for index in test:
        ls.append(run1(index, pruned_tree, labels))
    return ls

# 调用剪枝函数对决策树进行剪枝并进行分类
pruned_result1 = get_pruned_result(train1, test1, labels1)

生成决策树如下: {'纹理': {'清晰': {'根蒂': {'稍蜷': '是', '蜷缩': '是', '硬挺': '否'}}, '模糊': '否', '稍糊': {'色泽': {'青绿': '否', '浅白': '否', '乌黑': {'敲声': {'浊响': '是', '沉闷': '否'}}}}}}
剪枝后的决策树如下: {'纹理': {'清晰': {'根蒂': {'稍蜷': '是', '蜷缩': '是', '硬挺': '否'}}, '模糊': '否', '稍糊': {'色泽': {'青绿': '否', '浅白': '否', '乌黑': {'敲声': {'浊响': '是', '沉闷': '否'}}}}}}


- accuracy函数用于计算分类准确率。post_pruning2函数实现了后剪枝操作，其中validation_data是用于验证剪枝效果的数据集。

- 剪枝后的决策树pruned_tree可以用于进行分类预测，可以使用run2函数对新的数据进行分类。

In [12]:
def rate(data, predict):
    num = 0
    for i in range(len(data)):
        if data[i][-1] == predict[i]:
            num +=1
    return format(num / len(data), '.2%')
print("ID3分类器对于test1数据集的准确率是：", rate(test1, result1))

ID3分类器对于test1数据集的准确率是： 70.00%


### 计算准确率
可以看出，用这种方法得到树分类树对于小规模数据的分类效果相对较好。


In [13]:
def fen2(data):
    ls = data[:]
    ls.sort()
    result = []
    for i in range(len(ls) - 1):
        result.append((data[i + 1] + data[i]) / 2)
    return result

## C4.5算法
- C4.5算法与ID3算法相似，其对ID3算法进行了改进。
- 信息增益作为划分准则存在的问题：

     信息增益偏向于选择取值较多的特征进行划分。⽐如学号这个特征，每个学生都有一个不同的学号，如果根据学号对样本进行分类，则每个学生都属于不同的类别，这样是没有意义的。而C4.5在生成过程中，用**信息增益比**来选择特征，可以校正这个问题。
     
- 特点
  - 能够完成对连续属性的离散化处理
  - 能够对不完整数据进行处理
  - 需要对数据集进行多次的顺序扫描和排序

In [14]:
def fen3(data, index, kind, method):
    ls = []
    if method == 0:
        for temp in data:
            if temp[index] <= kind:
                t = temp[0 : index]
                t = t + temp[index + 1 : ]
                ls.append(t)
    else:
        for temp in data:
            if temp[index] > kind:
                t = temp[0 : index]
                t = t + temp[index + 1 : ]
                ls.append(t)
    return ls

## 利用$C4.5$算法构建决策树
信息增益作为划分准则存在的问题：<br/>
信息增益偏向于选择取值较多的特征进行划分。⽐如学号这个特征，每个学生都有一个不同的学号，如果根据学号对样本进行分类，则每个学生都属于不同的类别，这样是没有意义的。而C4.5在生成过程中，用信息增益比来选择特征，可以校正这个问题。<br/>
信息增益比 = 惩罚参数 * 信息增益，即 $g_R(D,A) = \frac{g(D,A)}{H_A(D)}$，其中的$H_A(D)$，对于样本集合D，将当前特征A作为随机变量（取值是特征A的各个特征值），求得的经验熵。<br/>
信息增益比本质： 是在信息增益的基础之上乘上一个惩罚参数。特征个数较多时，惩罚参数较小；特征个数较少时，惩罚参数较大。<br/>
惩罚参数：数据集D以特征A作为随机变量的熵的倒数，即：将特征A取值相同的样本划分到同一个子集中，$惩罚参数 = \frac{1}{H_A(D)}=\frac{1}{-\sum_{(i = 1)}^{n}\frac{|D_i|}{|D|}\log_2\frac{|D_i|}{|D|}}$ <br/>
- 将连续属性离散化<br/>
采用二分法，把数据由小到大排列，选择每个区间的中位点位取值，左侧区间为不大于该点的值，右侧区间为大于该点的值
- 将数据按照某一种类的属性或者数值的区间重新分类；method = 0 按照区间左侧分类，method = 1 按照区间右侧分类

In [15]:
def zuijiatezhengfenli2(data):
    base, mm= information(data) #原始的信息熵
    info = []
    for j in range(len(data[0]) - 1):
        dic = {}
        for i in data:
            current = i[j] #取出最后的结果
            if current not in dic.keys(): 
                dic[current] = 1 #创建一个新的类别
            else:
                dic[current] += 1 #原有类别+1
        result = 0.0
        for key in dic:
            prob = float(dic[key]) / len(data)
            result -= prob * math.log(prob,2) 
        info.append(result)
    best = 0
    bestindex = -1
    bestpartvalue = None #如果是离散值，使用该值进行分割
    for i in range(len(data[0]) - 1):
        #抽取该列数据的所有信息
        ls = [index[i] for index in data]
        feture = set(ls) 
        #计算该列的信息增益
        temp = 0.0
        if type(ls[0]) == type("a"):#判断是否时离散的
            for value in feture:
                datatemp = split(data, i, value)
                prob = len(datatemp) / float(len(data))
                t, mm = information(datatemp)
                temp += prob * t
        else:
            ls.sort()
            min = float("inf")
            for j in range(len(ls) - 1):
                part = (ls[j + 1] + ls[j]) / 2 #计算划分点
                left = split2(data, i, part, 0)
                right = split2(data, i, part, 1)
                temp1, useless = information(left)
                temp2, useless = information(right)
                temp = len(left) / len(data) * temp1 + len(right) / len(data) * temp2
                if temp < min:
                    min = temp
                    bestpartvalue = part
            temp = min
        infoGain = base - temp
        #根据信息增益挑选 
        if info[i] != 0:
            if infoGain / info[i] >= best:
                best = infoGain / info[i]
                bestindex = i
    return bestindex, bestpartvalue


## 对测试集利用$C4.5$决策树进行分类
- 取出当前树的根节点
- 利用跟节点信息查询当前输入数据输入内容
- 如果是离散值
    1. 如果查询出来的分支是叶节点，返回该值
    2. 如果不是叶节点，递归查询子树
- 如果是非离散值
    1. 取出节点值与现在数据进行比较
    2. 如果大于以“否”为标签，否则以“是”为标签
    3. 如果查询出来的分支是叶节点，返回该值
    4. 如果不是叶节点，递归查询子树

In [21]:
def fen4(data, labels):
    typelist = [index[-1] for index in data] #取出该数据集的分类
    nothing, typecount = information(data) #计算出类别种类以及数量
    if typecount == len(typelist): #如果只有一个类别
        return typelist[0]
    bestindex, part = chooseBestFeatureToSplit2(data)  # 最优划分属性的索引
    if bestindex == -1:
        return "是"
    if type([t[bestindex] for t in data][0]) == type("a"):#判断是否时离散的
        bestlabel = labels[bestindex]
        Tree = {bestlabel: {}}
        temp = copy.copy(labels)
        feature = [example[bestindex] for example in data]
        del (temp[bestindex])  # 已经选择的特征不再参与分类，将该类别删除
        unique = set(feature)  # 该属性所有可能取值，也就是节点的分支
        for i in unique:  
            s = temp[:] 
            Tree[bestlabel][i] = classify2(split(data, bestindex, i), s)
    else: #连续的变量
        bestlabel = labels[bestindex] + "<" + str(part)
        Tree = {bestlabel: {}}
        temp = labels[:]
        del(temp[bestindex])
        leftdata = split2(data, bestindex, part, 0)
        Tree[bestlabel]["是"] = classify2(leftdata, temp)
        rightdata = split2(data, bestindex, part, 1)
        Tree[bestlabel]["否"] = classify2(rightdata, temp)
    return Tree

- 在构建决策树的过程中，我们进行了预剪枝判断，即计算划分后的准确率，并与原始准确率进行比较，决定是否进行划分。如果划分后的准确率低于原始准确率，则进行剪枝，返回叶子节点。
- 剪枝后的决策树 pruned_tree 可以用于进行分类预测，可以使用 run2函数对新的数据进行分类。

In [22]:
def fen5(data, labels, validation_data=None, validation_labels=None):
    typelist = [index[-1] for index in data]  # 取出该数据集的分类
    nothing, typecount = information(data)  # 计算出类别种类以及数量
    if typecount == len(typelist):  # 如果只有一个类别
        return typelist[0]
    
    # 判断是否需要进行划分
    if validation_data is not None and validation_labels is not None:
        original_accuracy = accuracy(validation_data, validation_labels)
        bestindex, part = zuijiatezhengfenli2(data)
        if bestindex == -1:
            return "是"
        if type([t[bestindex] for t in data][0]) == type("a"):
            bestlabel = labels[bestindex]
            Tree = {bestlabel: {}}
            temp = copy.copy(labels)
            feature = [example[bestindex] for example in data]
            del(temp[bestindex])
            unique = set(feature)
            for i in unique:
                s = temp[:]
                Tree[bestlabel][i] = classify2(split(data, bestindex, i), s, validation_data, validation_labels)
        else:
            bestlabel = labels[bestindex] + "<" + str(part)
            Tree = {bestlabel: {}}
            temp = labels[:]
            del(temp[bestindex])
            leftdata = split2(data, bestindex, part, 0)
            Tree[bestlabel]["是"] = classify2(leftdata, temp, validation_data, validation_labels)
            rightdata = split2(data, bestindex, part, 1)
            Tree[bestlabel]["否"] = classify2(rightdata, temp, validation_data, validation_labels)
        
        # 计算划分后的准确率，并与原始准确率进行比较
        pruned_accuracy = accuracy(validation_data, validation_labels)
        if pruned_accuracy >= original_accuracy:
            return Tree  # 不剪枝，保持划分
        else:
            return typelist[0]  # 剪枝，返回叶子节点
    else:
        bestindex, part = zuijiatezhengfenli2(data)
        if bestindex == -1:
            return "是"
        if type([t[bestindex] for t in data][0]) == type("a"):
            bestlabel = labels[bestindex]
            Tree = {bestlabel: {}}
            temp = copy.copy(labels)
            feature = [example[bestindex] for example in data]
            del(temp[bestindex])
            unique = set(feature)
            for i in unique:
                s = temp[:]
                Tree[bestlabel][i] = fen4(split(data, bestindex, i), s)
        else:
            bestlabel = labels[bestindex] + "<" + str(part)
            Tree = {bestlabel: {}}
            temp = labels[:]
            del(temp[bestindex])
            leftdata = split2(data, bestindex, part, 0)
            Tree[bestlabel]["是"] = fen4(leftdata, temp)
            rightdata = split2(data, bestindex, part, 1)
            Tree[bestlabel]["否"] = fen4(rightdata, temp)
        return Tree



- 计算信息熵和信息增益并返回最优的分类方式，对数据的每项指标做以下的过程
    1. 抽取该项数据的所有信息
    2. 计算每一类的信息熵
    3. 按照该项数据的类别信息将数据集划分成多个子数据集
    4. 计算每个数据集的信息熵
    5. 计算该项数据的信息增益比
    6. 根据信息增益选择最好的分类项目，返回该项目的类别号
    7. 对连续型数据，计算分割值，求出信息增益比最小的点作为分割点返回

In [30]:
def ce2(data, tree, labels):
    firstStr = list(tree.keys())[0]  # 根节点
    firstLabel = firstStr
    t = str(firstStr).find('<') #查看是否是连续型
    if t > -1:  # 如果是连续型的特征
        firstLabel = str(firstStr)[ : t]
    secondDict = tree[firstStr]
    featIndex = labels.index(firstLabel)  # 跟节点对应的属性
    result = ''
    for key in list(secondDict.keys()):  # 对每个分支循环
        if type(data[featIndex]) == type("a"):
            if data[featIndex] == key:  # 测试样本进入某个分支
                if type(secondDict[key]).__name__ == 'dict':  # 该分支不是叶子节点，递归
                    result = run2(data, secondDict[key], labels)
                else:  # 如果是叶子， 返回结果
                    result = secondDict[key]
        else:
            value = float(str(firstStr)[t + 1 : ])
            if data[featIndex] <= value:
                if type(secondDict['是']).__name__ == 'dict':  # 该分支不是叶子节点，递归
                    result = run2(data, secondDict['是'], labels)
                else:  # 如果是叶子， 返回结果
                    result = secondDict['是']
            else:
                if type(secondDict['否']).__name__ == 'dict':  # 该分支不是叶子节点，递归
                    result = run2(data, secondDict['否'], labels)
                else:  # 如果是叶子， 返回结果
                    result = secondDict['否']
    return result

def accuracy(validation_data, validation_labels):
    correct = 0
    for i in range(len(validation_data)):
        if validation_data[i] == validation_labels[i]:
            correct += 1
    return correct / len(validation_data)
def shengc2(train, test, labels):
    ls = []   
    tree = fen4(train, labels)
    # 使用预剪枝方法对决策树进行构建和剪枝
    #pruned_tree = classify2(train2, labels2, validation_data=test2, validation_labels=labels2)
    print("生成决策树如下:", tree)
    for index in test:
        ls.append(run2(index, tree, labels))
        #ls2.append(run2(index,pruned_tree,labels))
    return ls
def shengc3(train, test, labels):
    ls2 = []
    #tree = classify2(train, labels)
    # 使用预剪枝方法对决策树进行构建和剪枝
    pruned_tree = fen4(train2, labels2, validation_data=test2, validation_labels=labels2)
    print("生成决策树如下:", pruned_tree)
    for index in test:
        #ls.append(run2(index, tree, labels))
        ls2.append(run2(index,pruned_tree,labels))
    return ls2
labels2 = ["色泽", "根蒂", "敲声", "纹理", "密度", "好瓜"]
result2 = shengc2(train2, test2, labels2)
result3 = shengc3(train2, test2, labels2)

生成决策树如下: {'纹理': {'清晰': {'根蒂': {'稍蜷': {'密度<0.3815': {'是': '是', '否': {'色泽': {'青绿': '是', '乌黑': '是'}}}}, '蜷缩': {'密度<0.5820000000000001': {'是': '是', '否': {'敲声': {'浊响': {'色泽': {'青绿': '是', '乌黑': '是'}}, '沉闷': {'色泽': {'青绿': '是', '乌黑': '是'}}}}}}, '硬挺': '是'}}, '模糊': {'密度<0.29400000000000004': {'是': '是', '否': '是'}}, '稍糊': {'敲声': {'浊响': {'密度<0.56': {'是': '是', '否': '是'}}, '沉闷': {'密度<0.6615': {'是': '是', '否': {'根蒂': {'稍蜷': '是', '蜷缩': '是'}}}}}}}}
生成决策树如下: {'纹理': {'清晰': {'根蒂': {'稍蜷': {'密度<0.3815': {'是': '是', '否': {'色泽': {'青绿': '是', '乌黑': '是'}}}}, '蜷缩': {'密度<0.5820000000000001': {'是': '是', '否': {'敲声': {'浊响': {'色泽': {'青绿': '是', '乌黑': '是'}}, '沉闷': {'色泽': {'青绿': '是', '乌黑': '是'}}}}}}, '硬挺': '是'}}, '模糊': {'密度<0.29400000000000004': {'是': '是', '否': '是'}}, '稍糊': {'敲声': {'浊响': {'密度<0.56': {'是': '是', '否': '是'}}, '沉闷': {'密度<0.6615': {'是': '是', '否': {'根蒂': {'稍蜷': '是', '蜷缩': '是'}}}}}}}}


# 递归构建$C4.5$决策树
- 决策树的生成
	1. 从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	2. 在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
- 迭代停止条件
    1. 当前结点所有样本都属于同⼀类别；
    2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
    3. 达到最大树深；
    4. 达到叶子结点的最小样本数；
- 具体实现
    1. 首先取出该数据集的类别信息
    2. 统计处类别信息以及数量，如果只有一个类别，返回该类别
    3. 计算最优划分的索引
    4. 初始化子树
    5. 当前已经选择的特征如果是离散的便不再参与分类，将该特征删除；如果是连续的不删除该特征，以分界点构建左子树和右子树
    6. 计算剩余特征的集合
    7. 对于每个分支，进行递归
    8. 返回值为子树

In [31]:
print("C4.5分类器对于test2数据集的准确率是：", rate(test2, result2))
print("C4.5剪枝分类器对于test2数据集的准确率是：", rate(test2, result3))

C4.5分类器对于test2数据集的准确率是： 60.00%
C4.5剪枝分类器对于test2数据集的准确率是： 60.00%


### 计算准确率
可以看出，C4.5预测的准确率稍较为一般

# 总结
本次实验进行了决策树的判断以及决策树构造剪枝等等的编程工作，我们在使用过常见的调库解决决策树问题之后采用现在的原始方法构造解决决策树问题，能够更加深刻的领悟到决策树算法的原理，对决策树的应用也能更加的得心应手，面对决策树问题的时候也能够更加的熟练。同时为大作业的编程提供了一定的参考意义。
本次实验当中采用了剪枝算法，我们构造的决策树经历了剪枝之后变化不大，原因可能是构造的决策树较小，样本集仍然不够大，需要我们不断的扩展决策树，才能够更好的展现决策树的实现价值。